In [2]:
!pip install circlify
!pip install chromatose

In [1]:
import numpy as np
import pandas as pd
import biocircuits 
import scipy.integrate
import scipy.signal

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p):
    p.outline_line_width=1.0
    p.outline_line_color="black"
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.visible=False
    p.yaxis.visible=False
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    p.toolbar.autohide=True
    p.toolbar_location=None
    
    return p

Loading BokehJS ...

In [2]:
import circlify as circ
import chromatose as ct

# more 3.2 

remaining ideas:
- ~color of new points matches the last point in the trajectories from the previous generation~
- bokeh animation?... 
- explore more circle packing algorithms

In [10]:
def deriv(X, t, alpha, beta, gamma, kappa):
    return alpha + beta * X / (1+X) - X - gamma * X / (1+kappa*X)

def retrieve_X_trajs(Xos, t, alphas, betas, gammas, kappas):
    if type(alphas) == float: 
        alphas = [alphas]*len(t)
        betas = [betas]*len(t)
        gammas = [gammas]*len(t)
        kappas = [kappas]*len(t)
        
    X_trajs = []
    for Xo, alpha, beta, gamma, kappa in zip(Xos, alphas, betas, gammas, kappas): 
        args = (alpha, beta, gamma, kappa)
        X_traj = scipy.integrate.odeint(deriv, Xo, t, args=args).T[0]
        X_trajs.append(list(X_traj))
    return X_trajs

def generate(Xos, t, n_generations, args):
    signals = []
    for _ in range(n_generations):
        X_trajs = retrieve_X_trajs(Xos, t, *args)
        final_concs = [traj[-1] for traj in X_trajs]
        signals.append(final_concs)
        Xos = final_concs
    signals = np.array(signals).T

    return signals

In [11]:
def noise(n_start, n_final):
    np.random.seed(123456789)
    
    xold = np.linspace(0,1, n_start)
    yold = -2*np.random.rand(n_start)+1
    
    xnew = np.linspace(0,1, n_final)
    f = scipy.interpolate.interp1d(xold, yold, kind="cubic")
    ynew = f(xnew)
    
    return ynew

In [5]:
def color_mapper(val, signal_min, signal_max):
    
    if val >= signal_max: val = signal_max
    if val <= signal_min: val = signal_min
        
    percentage = (val - signal_min) / (signal_max-signal_min)

    index = int(percentage*len(palette[:-2]))
    
    return palette[index]

def df_signal(signals, 
              n_mothers, n_generations, 
              signal_min, signal_max, 
              automap=False):

    # mother column... 
    _mother_column = []
    for i in range(n_mothers):
        _mother_column += [i]*n_generations

    # signal column... 
    _signal_column = [signal for group in signals for signal in group]
    
    # color column... 
    if automap: 
        signal_min = min(_signal_column)
        signal_max = max(_signal_column)
        
    _color_column = [ color_mapper(_, signal_min, signal_max) 
                          for _ in _signal_column ]

    # generation column... 
    _generation_column = [_ for _ in range(n_generations)] * n_mothers


    df = pd.DataFrame({'mother': _mother_column,
                       'generation': _generation_column,
                       'signal': _signal_column,
                       'color': _color_column,
                     })
    return df

def df_position(
        df, 
        n_mothers, n_generations, 
        colony_radius, enclosure_radius, 
        jitter, jitter2
    ):
    # ................ PACK SINGLE COLONY ................
    _circles = circ.circlify( [12]*(n_generations) ) 
    _xs = [c.x for c in _circles] * n_mothers
    _ys = [c.y for c in _circles] * n_mothers

    df["x"] = np.array(_xs) * colony_radius
    df["y"] = np.array(_ys) * colony_radius


    # ................ PACK MOTHER CELLS ................
    _circles = circ.circlify( [12]*(n_mothers) )  
    _xs = [c.x for c in _circles for i in range(n_generations)]
    _ys = [c.y for c in _circles for i in range(n_generations)]

    np.random.seed(123456789)
    
#     jitter_mother_x = noise(50, len(df))*jitter2
#     jitter_mother_y = noise(50, len(df))*jitter2
    jitter_mother_x = (-2*np.random.rand(n_mothers)+1)*jitter2
    jitter_mother_y = (-2*np.random.rand(n_mothers)+1)*jitter2
    jitter_mother_x = [__ for _ in jitter_mother_x for __ in [_]*n_generations]
    jitter_mother_y = [__ for _ in jitter_mother_y for __ in [_]*n_generations]

    df["x"] += (np.array(_xs)+jitter_mother_x) * enclosure_radius
    df["y"] += (np.array(_ys)+jitter_mother_y) * enclosure_radius

#     # ................ JITTERING ................
    df["x"] += (-2*np.random.rand(len(df))+1)*jitter
    df["y"] += (-2*np.random.rand(len(df))+1)*jitter

    return df


def dataframer(
        args, Xos,
        n_mothers, n_generations, 
        signal_min, signal_max, palette, 
        colony_radius, enclosure_radius, jitter, jitter2,
        automap=False
    ):
    signals = generate(Xos, t, n_generations, args)
    df = df_signal(signals, n_mothers, n_generations, 
                   signal_min, signal_max, automap=automap)
    df = df_position(df, n_mothers, n_generations, 
                     colony_radius, enclosure_radius, jitter, jitter2)
    return df

In [6]:
palette = ct.palpolate(["#1c2641", "#808BBE","#D4B3CF"])
t = np.linspace(0, 10, 500)

# wDIGet tmIME ⏰

In [18]:
n_mothers = 3
n_generations = 5

colony_radius = 0.001
jitter = 0.1
jitter2 = 0.5 #0.1
signal_min = -2.0                            # colormapper min signal value
signal_max = 3.3                             # colormapper max signal value
    
enclosure_radius = 1
Xos = np.linspace(0, 5, n_mothers)
alpha = 1.3
beta = 11
gamma = 25 
kappa = 3.5
noise_percent = 5
freq_offset, phase_offset = 1, 5
alphas = np.abs(noise_percent/100*alpha * np.sin(t) + alpha)
betas = np.abs(noise_percent/100*beta * np.sin(freq_offset*t+0.5*phase_offset) + beta)
gammas = np.abs(noise_percent/100*gamma * np.sin(freq_offset*t+1.0*phase_offset) + gamma)
kappas = np.abs(noise_percent/100*kappa * np.sin(freq_offset*t+1.5*phase_offset) + kappa)

args = (alphas, betas, gammas, kappas)
df = dataframer(
        args, Xos, 
        n_mothers, n_generations, 
        signal_min, signal_max, 
        palette, colony_radius, enclosure_radius, jitter, jitter2,
        automap=True
    )

In [19]:
df

,mother,generation,signal,color,x,y
0,0,0,4.388141,#1c2641,0.354826,-0.561976
1,0,1,4.421308,#d2b2ce,0.499955,-0.445164
2,0,2,4.421340,#d2b2ce,0.397159,-0.462844
3,0,3,4.421340,#d2b2ce,0.403246,-0.577565
4,0,4,4.421340,#d3b2ce,0.385876,-0.489926
5,1,0,4.410469,#9c98c3,-0.547650,0.026016
6,1,1,4.412730,#a79dc6,-0.562281,0.024026
7,1,2,4.412732,#a79dc6,-0.417820,-0.037979
8,1,3,4.412732,#a79dc6,-0.398282,0.048384
9,1,4,4.412732,#a79dc6,-0.445067,-0.100200


In [25]:
n_mothers_slider = pn.widgets.IntSlider(name="# mothers", start=10, end=30, value=10, width=240)
n_generations_slider = pn.widgets.IntSlider(name="# generations", start=10, end=30, value=28, width=240)
generation_slider = pn.widgets.IntSlider(name="generation", start=0, end=50, value=0, width=400)

colony_radius_slider = pn.widgets.FloatSlider(name="colony radius", start=1, end=20, value=10, width=240, step=0.1)
enclosure_radius_slider = pn.widgets.FloatSlider(name="enclosure radius", start=1, end=1.1, value=1, width=240, step=0.01)
jitter_slider = pn.widgets.FloatSlider(name="jitter ", start=0.0, end=1, value=0.7, step=0.1, width=240)
jitter2_slider = pn.widgets.FloatSlider(name="jitter 2", start=0.0, end=2, value=0.7, step=0.1, width=240)
point_size_slider = pn.widgets.FloatSlider(name="point size", start=10, end=20, value=15, width=240, step=1)

@pn.depends(generation_slider.param.value,
            n_mothers_slider.param.value,
            n_generations_slider.param.value,
            colony_radius_slider.param.value, 
            enclosure_radius_slider.param.value, 
            jitter_slider.param.value, 
            jitter2_slider.param.value,
            point_size_slider.param.value,)
def plotter(generation, 
            n_mothers, n_generations, 
            colony_radius, enclosure_radius, jitter, jitter2, point_size
           ):
    colony_radius = 0.001
    jitter = 0.1
    jitter2 = 0.5 #0.1
    signal_min = -2.0                            # colormapper min signal value
    signal_max = 3.3                             # colormapper max signal value

    # deterministic
    Xos = [1] * n_mothers                        # initial distribution of signals
    Xos = np.linspace(0, 5, n_mothers)
    t = np.linspace(0, 10, 500)

    alpha = 1.3
    beta = 11
    gamma = 25 
    kappa = 3.5
    args = (alpha, beta, gamma, kappa)
    df = dataframer(
            args, Xos,
            n_mothers, n_generations, 
            signal_min, signal_max, 
            palette, colony_radius, enclosure_radius, jitter, jitter2
        )
    
    sub_df = df.loc[df['generation']<=generation]
    p = bokeh.plotting.figure(height=400, width=400,  title="Deterministic")
    p.circle(source=df, x="x", y="y",color="white", size=point_size, alpha=0.0)
    p.circle(source=sub_df, x="x", y="y",color="color", size=point_size, 
                 line_color="black",line_width=0.3,)

    signal_min = 4.3                             # colormapper min signal value
    signal_max = 4.4                             # colormapper max signal value

    # gaussian
    noise_percent = 5
    freq_offset, phase_offset = 1, 5
    alphas = np.abs(noise_percent/100*alpha * np.sin(t) + alpha)
    betas = np.abs(noise_percent/100*beta * np.sin(freq_offset*t+0.5*phase_offset) + beta)
    gammas = np.abs(noise_percent/100*gamma * np.sin(freq_offset*t+1.0*phase_offset) + gamma)
    kappas = np.abs(noise_percent/100*kappa * np.sin(freq_offset*t+1.5*phase_offset) + kappa)

    args = (alphas, betas, gammas, kappas)
    df = dataframer(
            args, Xos, 
            n_mothers, n_generations, 
            signal_min, signal_max, 
            palette, colony_radius, enclosure_radius, jitter, jitter2,
            automap=True
        )
    sub_df = df.loc[df['generation']<=generation]
    q = bokeh.plotting.figure(height=400, width=400, title='Stochastic ')
    q.circle(source=df, x="x", y="y",color="white", size=point_size, alpha=0.0)
    q.circle(source=sub_df, x="x", y="y",color="color", size=point_size, 
                 line_color="black",line_width=0.3)
    return pn.Row(style(p), style(q))

In [26]:
lay_widgets = pn.Row(pn.Column(colony_radius_slider, point_size_slider), 
                     pn.Column(jitter_slider, jitter2_slider), align="center")
lay_generation = pn.Row(generation_slider, align="center")
lay_control = pn.Row(n_mothers_slider, n_generations_slider, align="center")
pn.Column(lay_generation, lay_control,  plotter, lay_widgets)

Column
    [0] Row(align='center')
        [0] IntSlider(end=50, name='generation', width=400)
    [1] Row(align='center')
        [0] IntSlider(end=30, name='# mothers', start=10, value=10, width=240)
        [1] IntSlider(end=30, name='# generations', start=10, value=28, width=240)
    [2] ParamFunction(function)
    [3] Row(align='center')
        [0] Column
            [0] FloatSlider(end=20, name='colony radius', start=1, value=10, width=240)
            [1] FloatSlider(end=20, name='point size', start=10, step=1, value=15, width=240)
        [1] Column
            [0] FloatSlider(name='jitter ', value=0.7, width=240)
            [1] FloatSlider(end=2, name='jitter 2', value=0.7, width=240)

In [9]:
generation = 0
n_mothers, n_generations = 30, 50
point_size = 15
enclosure_radius = 1
colony_radius = 0.001
jitter = 0.1
jitter2 = 0.5 #0.1
signal_min = -2.0                            # colormapper min signal value
signal_max = 3.3                             # colormapper max signal value

# deterministic
Xos = [1] * n_mothers                        # initial distribution of signals
Xos = np.linspace(0, 5, n_mothers)
t = np.linspace(0, 10, 500)

alpha = 1.3
beta = 11
gamma = 25 
kappa = 3.5
args = (alpha, beta, gamma, kappa)
dfP = dataframer(
        args, Xos,
        n_mothers, n_generations, 
        signal_min, signal_max, 
        palette, colony_radius, enclosure_radius, jitter, jitter2
    )
sub_dfP = dfP.loc[dfP['generation']<=generation]
p = bokeh.plotting.figure(height=400, width=400,  title="Deterministic")

sourceP = bokeh.models.ColumnDataSource({'x':dfP.x.values, 'y':dfP.y.values,})
sub_sourceP = bokeh.models.ColumnDataSource({'x':sub_dfP.x.values, 'y':sub_dfP.y.values,})
p.circle(source=sourceP, x="x", y="y",color="white", size=point_size, alpha=0.0)
p.circle(source=sub_sourceP, x="x", y="y",color="color", size=point_size, 
             line_color="black",line_width=0.3,)

signal_min = 4.3                             # colormapper min signal value
signal_max = 4.4                             # colormapper max signal value

# gaussian
noise_percent = 5
freq_offset, phase_offset = 1, 5
alphas = np.abs(noise_percent/100*alpha * np.sin(t) + alpha)
betas = np.abs(noise_percent/100*beta * np.sin(freq_offset*t+0.5*phase_offset) + beta)
gammas = np.abs(noise_percent/100*gamma * np.sin(freq_offset*t+1.0*phase_offset) + gamma)
kappas = np.abs(noise_percent/100*kappa * np.sin(freq_offset*t+1.5*phase_offset) + kappa)

args = (alphas, betas, gammas, kappas)
dfQ = dataframer(
        args, Xos, 
        n_mothers, n_generations, 
        signal_min, signal_max, 
        palette, colony_radius, enclosure_radius, jitter, jitter2,
        automap=True
    )
sub_dfQ = df.loc[dfQ['generation']<=generation]
q = bokeh.plotting.figure(height=400, width=400, title='Stochastic ')

sourceQ = bokeh.models.ColumnDataSource({'x':dfQ.x.values, 'y':dfQ.y.values,})
sub_sourceQ = bokeh.models.ColumnDataSource({'x':sub_dfQ.x.values, 'y':sub_dfQ.y.values,})

q.circle(source=sourceQ, x="x", y="y",color="white", size=point_size, alpha=0.0)
q.circle(source=sub_sourceQ, x="x", y="y",color="color", size=point_size, 
             line_color="black",line_width=0.3)

NameError: name 'df' is not defined

In [ ]:
df

In [10]:
from bokeh.io import curdoc
from bokeh.client import push_session

def animate_update():
    generation = generation_slider.value + 1
    if generation >= 30:
        generation = 0
    generation_slider.value = generation
    
def callback(attrname, old, new):
    g = generation_slider.value
    sub_dfP = dfP.loc[dfP['generation']<=g]
    sub_dfQ = dfQ.loc[dfQ['generation']<=g]
    
    sub_sourceP.data["x"] = sub_dfP.x.values
    sub_sourceP.data["y"] = sub_dfP.y.values
    
    sub_sourceQ.data["x"] = sub_dfQ.x.values
    sub_sourceQ.data["y"] = sub_dfQ.y.values
    
layout = bokeh.layouts.layout([[p, q]])
curdoc().add_root(layout)
curdoc().add_periodic_callback(animate_update, 30)

# session.show(p) # open the document in a browser
# session.loop_until_closed() # run forever

NameError: name 'q' is not defined

In [ ]:
def animate_update():
    time = slider.value + 1
    if time >= 30:
        time = 0
    slider.value = time

def slider_update(attrname, old, new):
    plot.update(slider.value)
    
slider = Slider(start=0, end=30, value=0, step=1, title="t")
slider.on_change('value', slider_update)


# Combine the bokeh plot on plot.state with the widgets
layout = layout([
    [plot.state],
    [slider]
], sizing_mode='fixed')

curdoc().add_root(layout)

curdoc().add_periodic_callback(animate_update, 1)

In [ ]:


from numpy import pi, cos, sin, linspace, roll
from bokeh.plotting import figure

# open a session to keep our local document in sync with server
session = push_session(curdoc())

ds = r.data_source

def update():
    rmin = roll(ds.data["inner_radius"], 1)
    rmax = roll(ds.data["outer_radius"], -1)
    ds.data.update(inner_radius=rmin, outer_radius=rmax)

curdoc().add_periodic_callback(update, 30)

session.show(p) # open the document in a browser

session.loop_until_closed() # run forever